# Train Tensorflow Model for Text classification

In this notebook we will be creating Tensorflow model for text classifcation which we classy user's queries to appropriate response tags.
The model will be saved to be used in the chat box



In [1]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf

## Data

`intents.json` file contains data with input patterns and expected replys, a Tensorflow based classifer  is used to classify the text input from user to these tags and then a random reply is selected for the user


Here is an example


`
    {"tag":"tumor",
    "patterns": ["what is tumor?", "What is lump?", "What is benign tumor?", "What is malignant tumor" ],
    "responses":["A tumor or lump is any abnormal mass of tissue or swelling. Like a cyst, a tumor can form in any part of the body. A tumor can be benign or cancerous (malignant), A benign tumor has distinct, smooth, regular borders. A malignant tumor has irregular borders and grows faster than a benign tumor. \n A malignant tumor can also spread to other parts of your body. A benign tumor can become quite large, but it will not invade nearby tissue or spread to other parts of your body."]
    },`
    
    
All the responses and pattens are related to Breast cancer and have been taken from online cancer forums and other cancer related sites.

Finally the classification model will be saved and will be used by Chatbot App in `Main Breast Cancer Helper Chatbox` jupyter notebook


In [2]:
#load json data
intents = json.loads(open('data/chatbot_intents/intents.json').read())

In [3]:
complete_data = {}
complete_data["tag"] =[]
complete_data["patterns"] = []


In [4]:
for intent in intents["intents"]:
    tag =  intent['tag']
    resp = intent['responses']
    for pat in intent["patterns"]:
        complete_data["tag"].append(tag)
        complete_data["patterns"].append(pat)
        

In [5]:
# convert intents data to Data frame
input_df = pd.DataFrame(complete_data)

In [6]:
# Encode the tags  to form classes
from sklearn.preprocessing import LabelEncoder
lbEncode = LabelEncoder()

input_df["class"] = lbEncode.fit_transform(input_df["tag"])

In [7]:
input_df.head()

,tag,patterns,class
0,greetings,hello,11
1,greetings,hey,11
2,greetings,hi,11
3,greetings,hi buddy,11
4,greetings,good day,11


`input_df` contains inputs  `pattens` and `class`. `class` is created by label encoding `tag`.
`pattens` are question which can be asked by user. Chatbot doesn't need user to add exactly same questions given in `pattern` but they are used for training classifier.


In [8]:
output_data = {}
output_data["tag"] =[]
output_data["responses"] = []


for intent in intents["intents"]:
    output_data["tag"].append(intent['tag'])
    output_data["responses"].append(intent['responses'])

In [9]:
# create output data frame with responses
output_df = pd.DataFrame(output_data)


In [10]:
output_df["class"] =  lbEncode.transform(output_df["tag"])

In [11]:
output_df.head()

,tag,responses,class
0,greetings,"[Hello, Hey!, What can I do for you?]",11
1,goodbye,"[Talk to you later, Goodbye!]",10
2,cancerInfo,[A disease in which abnormal cells divide unco...,6
3,BreastcancerInfo,[When cells divide uncontrollably in Breast an...,0
4,tumor,[A tumor or lump is any abnormal mass of tissu...,17


`output_df` is contains all the fileds of `input_df` and one additional filed of `responses`.  Once the classifier classifies the class, one of the response from the corresponding responses will be taken and displayed to the user.

In [12]:
output_df.to_json("save_replys_dataframe/output_df.json")

In [13]:
# Number of classes
max(input_df['class'])+1

19

## Modelling

In [14]:
# download universal sentence encoder layer or text encoding from tensorflow hub
import tensorflow_hub as hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                       input_shape=[],
                                       dtype = "string",
                                       trainable = False,
                                       name="USE")

In [15]:
# Create a TF model for Text classification
from tensorflow.keras import layers
import tensorflow as tf

model = tf.keras.Sequential([
    sentence_encoder_layer, 
    layers.Reshape((1,512)),
    layers.LSTM(512),
    layers.Dense(64, activation = "relu"),
    layers.Dropout(rate=0.2),
    layers.Dense(64, activation = "relu"),
    layers.Dropout(rate=0.2),
    layers.Dense(max(input_df['class'])+1, activation = "softmax")
], name = "model_USE")

In [16]:
# compile and fit the model
model.compile(loss=tf.losses.sparse_categorical_crossentropy,
               optimizer='Adam',
               metrics = ["accuracy"])
                
model.fit(x=input_df['patterns'],
           y=input_df['class'],
           epochs=90)

Epoch 1/90
3/3 [==============================] - 0s 9ms/step - loss: 2.9402 - accuracy: 0.1463
Epoch 2/90
3/3 [==============================] - 0s 8ms/step - loss: 2.9134 - accuracy: 0.2561
Epoch 3/90
3/3 [==============================] - 0s 8ms/step - loss: 2.8799 - accuracy: 0.2927
Epoch 4/90
3/3 [==============================] - 0s 8ms/step - loss: 2.8344 - accuracy: 0.3171
Epoch 5/90
3/3 [==============================] - 0s 8ms/step - loss: 2.7923 - accuracy: 0.2805
Epoch 6/90
3/3 [==============================] - 0s 8ms/step - loss: 2.7443 - accuracy: 0.2317
Epoch 7/90
3/3 [==============================] - 0s 8ms/step - loss: 2.6308 - accuracy: 0.3049
Epoch 8/90
3/3 [==============================] - 0s 8ms/step - loss: 2.5370 - accuracy: 0.3537
Epoch 9/90
3/3 [==============================] - 0s 8ms/step - loss: 2.4231 - accuracy: 0.3415
Epoch 10/90
3/3 [==============================] - 0s 8ms/step - loss: 2.2904 - accuracy: 0.3171
Epoch 11/90
3/3 [======================

3/3 [==============================] - 0s 8ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 86/90
3/3 [==============================] - 0s 7ms/step - loss: 0.0534 - accuracy: 0.9878
Epoch 87/90
3/3 [==============================] - 0s 8ms/step - loss: 0.0402 - accuracy: 0.9878
Epoch 88/90
3/3 [==============================] - 0s 8ms/step - loss: 0.0276 - accuracy: 0.9878
Epoch 89/90
3/3 [==============================] - 0s 7ms/step - loss: 0.0543 - accuracy: 0.9878
Epoch 90/90
3/3 [==============================] - 0s 8ms/step - loss: 0.0419 - accuracy: 0.9878


## Save Model

In [17]:
# save the text classifier model
model.save("models/NLP_model.h5")


## Model summary

In [18]:
model.summary()

Model: "model_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
reshape (Reshape)            (None, 1, 512)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0 